Импорт необходимых библиотек:

In [77]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [ ]:
eps = 1e-8 # небольшая регуляризация для устройчивости

Чтение датасета:

In [81]:
df = pd.read_csv('data/WineQT.csv')

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


Удаление лишнего столбца, чтобы избежать ложных корреляций:

In [5]:
df = df.drop('Id', axis=1)

Разделение на выборку и таргет:

In [6]:
X = df.drop('quality', axis=1)
y = df['quality']

Сплит датасета на тренировочную и тестовую выборку:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Определение функции для генерации пропусков в данных (тип пропусков - MCAR, missing completely at random)

Данная функция работает по-разному в зависимости от количества передаваемых ей признаков, в которых нужны пропуски:


*   Один признак - считается общее кол-во пропущенных значений (num_missing), выбираются случайные объекты, после чего датафрейм заполняется пропусками, а маска - значениями True
*   Много признаков (> 1) - также считается общее кол-во ячеек, в которых будет стоять пропуск (только уже не в одном столбце, а в подматрице, заданной параметром features)



In [8]:
def generate_mcar(df, features, missing_rate=0.1, seed=42):
  rng = np.random.default_rng(seed)
  df_for_mcar = df.copy()
  if len(features) == 1:
    mask = pd.Series(False, index=df_for_mcar.index)
    df_for_mcar = df.copy()
    num_missing = int(df_for_mcar.shape[0] * missing_rate)
    missing_rows = rng.choice(df_for_mcar.index.to_numpy(), size=num_missing, replace=False)
    mask.loc[missing_rows] = True
    df_for_mcar.loc[missing_rows, features[0]] = np.nan
  elif len(features) > 1:
    features = np.array(features)
    mask = pd.DataFrame(np.full((df_for_mcar.shape[0], len(features)), False), index=df_for_mcar.index, columns=features)
    num_missing = int(df_for_mcar.shape[0] * len(features) * missing_rate) # кол-во пропусков
    n_cells = df_for_mcar.shape[0] * len(features) # общее кол-во ячеек в подматрице
    idxs = np.random.choice(n_cells, num_missing, replace=False) # выбираем num_missing ячеек из n_cells
    row_pos = idxs // len(features) # получаем позиции в строках
    col_pos = idxs % len(features) # получаем позиции в столбцах
    for r, c in zip(row_pos, col_pos): # по полученным координатам собираем датафрейм и маску
                mask.iat[r, c] = True
                df_for_mcar.loc[df_for_mcar.index[r], features[c]] = np.nan
  return df_for_mcar, mask

## 1) Пропуски в одном признаке:


Генерация пропусков:

In [9]:
X_test_with_nan, mask = generate_mcar(X_test, ['total sulfur dioxide'], missing_rate=0.1, seed=42)

Проверка наличия пропусков:

In [10]:
X_test_with_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229 entries, 158 to 966
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         229 non-null    float64
 1   volatile acidity      229 non-null    float64
 2   citric acid           229 non-null    float64
 3   residual sugar        229 non-null    float64
 4   chlorides             229 non-null    float64
 5   free sulfur dioxide   229 non-null    float64
 6   total sulfur dioxide  207 non-null    float64
 7   density               229 non-null    float64
 8   pH                    229 non-null    float64
 9   sulphates             229 non-null    float64
 10  alcohol               229 non-null    float64
dtypes: float64(11)
memory usage: 29.6 KB


#### В первую очередь, реализовывается baseline - заполенение пропусков значением **медианы/среднего**:

Вычисление медианы и среднего по обучающей выборке:

In [11]:
median_X_train = np.median(X_train['total sulfur dioxide'])
mean_X_train = np.mean(X_train['total sulfur dioxide'])

In [12]:
print(median_X_train)
print(mean_X_train)

38.0
45.838621444201316


Подготовка датафреймов:

In [13]:
X_test_mean_filled = X_test_with_nan.copy()
X_test_median_filled = X_test_with_nan.copy()

Заполнение датафреймов значениями медианы и среднего:

In [14]:
X_test_mean_filled['total sulfur dioxide'] = np.where(X_test_with_nan['total sulfur dioxide'].isna(), mean_X_train, X_test_with_nan['total sulfur dioxide'])
X_test_median_filled['total sulfur dioxide'] = np.where(X_test_with_nan['total sulfur dioxide'].isna(), median_X_train, X_test_with_nan['total sulfur dioxide'])

Получение метрики MAE для оценки заполненных пропусков:

*Использование MAE обусловлено выбросами в данных, а MAE более устойчиво отражает типичную ошибку восстановления

In [15]:
mae_with_mean = mean_absolute_error(X_test.loc[mask, 'total sulfur dioxide'], X_test_mean_filled.loc[mask, 'total sulfur dioxide'])
mae_with_median = mean_absolute_error(X_test.loc[mask, 'total sulfur dioxide'], X_test_median_filled.loc[mask, 'total sulfur dioxide'])

In [16]:
print(mae_with_mean)
print(mae_with_median)

20.318181818181817
20.681818181818183


#### Импутация пропусков на основе **условного распределения многомерной нормальной модели**:

Оценка параметров многомерного гауссовского распределения на train выборке:

In [17]:
m = np.mean(X_train, axis=0) # вектор средних по каждому признаку
sigma = np.cov(X_train, rowvar=False) # ковариационная матрица

In [82]:
# Выделяем "целевой" для импутации признак x_j, а также матрицу остальных наблюдаемых признаков x_obs:
x_j = X_train['total sulfur dioxide']
x_obs = X_train.drop('total sulfur dioxide', axis=1)

In [19]:
j = X_train.columns.to_list().index('total sulfur dioxide') # индекс соответстующего "целевого" признака

In [20]:
m_j = m.iloc[j] # среднее целевого признака
m_obs = np.delete(m, j) #средние по всем остальным признакам

In [21]:
sigma_obs_obs = np.delete(np.delete(sigma, j, axis=0), j, axis=1) # ковариация наблюдаемых признаков между собой
sigma_j_obs = np.delete(sigma[j, :], j) # ковариация целевого признака с каждым наблюдаемым признаком

In [22]:
X_obs = X_test_with_nan[mask].drop('total sulfur dioxide', axis=1) # оставляем только наблюдаемые признаки

In [23]:
Z = X_obs - m_obs # центрируем наблюдаемые признаки

In [25]:
A = sigma_obs_obs + eps * np.eye(sigma_obs_obs.shape[0]) # регуляризированная матрица
w = np.linalg.solve(A, sigma_j_obs) # решение линейной системы. Этот вектор коэффициентов задает вклад каждого наблюдаемого признака в восстановление x_j

In [26]:
pred = m_j + Z @ w # вычисление условного ожидания для пропущенных значений

In [27]:
X_test_gauss_filled = X_test_with_nan.copy()
X_test_gauss_filled.loc[mask, 'total sulfur dioxide'] = pred # записываем полученные предсказания в подготовленный датафрейм

Получение метрики MAE для оценки заполненных пропусков:

In [28]:
mae_with_gauss = mean_absolute_error(X_test.loc[mask, 'total sulfur dioxide'], X_test_gauss_filled.loc[mask, 'total sulfur dioxide'])

In [29]:
mae_with_gauss

11.728818382612538

### Вывод:  
Методы заполнения средним и медианой показали практически одинаковый уровень ошибки. Это ожидаемо, поскольку оба подхода являются константной импьютацией и используют только одномерную статистику признака, полностью игнорируя его связь с другими переменными.

Восстановление с помощью многомерной гауссовской модели дало существенно более низкую ошибку — почти в два раза меньше по сравнению с базовыми методами. Это связано с тем, что условное математическое ожидание учитывает ковариационную структуру данных и использует информацию из остальных признаков для предсказания пропущенного значения. Таким образом, модель опирается на линейные зависимости между переменными, что значительно повышает точность восстановления.

Полученные результаты подтверждают, что при наличии корреляций между признаками многомерная вероятностная модель обеспечивает более эффективную импьютацию по сравнению с методами, основанными только на центральных тенденциях.

## 2) Пропуски в множестве признаков

Генерация пропусков:

In [30]:
X_test_with_nan, mask = generate_mcar(X_test, X_test.columns, missing_rate=0.1, seed=42)

Отбор признаков, в которых попался хотя бы один пропуск:

In [31]:
features_with_nan = [i for i in X_test.columns if X_test_with_nan[i].isna().sum()]

Вычисление медианы и среднего по каждому признаку обучающей выборки:

In [32]:
mean_X_train = X_train[features_with_nan].mean()
median_X_train = X_train[features_with_nan].median()

In [33]:
print(mean_X_train)
print(median_X_train)

fixed acidity            8.258096
volatile acidity         0.531018
citric acid              0.265799
residual sugar           2.519092
chlorides                0.086476
free sulfur dioxide     15.706783
total sulfur dioxide    45.838621
density                  0.996683
pH                       3.314234
sulphates                0.655646
alcohol                 10.439187
dtype: float64
fixed acidity            7.900000
volatile acidity         0.520000
citric acid              0.250000
residual sugar           2.200000
chlorides                0.078000
free sulfur dioxide     13.000000
total sulfur dioxide    38.000000
density                  0.996645
pH                       3.310000
sulphates                0.620000
alcohol                 10.200000
dtype: float64


Подготовка датафреймов:

In [34]:
X_test_mean_filled = X_test_with_nan.copy()
X_test_median_filled = X_test_with_nan.copy()

Заполнение датафреймов значениями медианы и среднего:

In [35]:
for feature in features_with_nan:
  X_test_mean_filled.loc[mask[feature], feature] = mean_X_train[feature]

In [36]:
for feature in features_with_nan:
  X_test_median_filled.loc[mask[feature], feature] = median_X_train[feature]

Проверка заполненных пропусков:

In [37]:
X_test_mean_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229 entries, 158 to 966
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         229 non-null    float64
 1   volatile acidity      229 non-null    float64
 2   citric acid           229 non-null    float64
 3   residual sugar        229 non-null    float64
 4   chlorides             229 non-null    float64
 5   free sulfur dioxide   229 non-null    float64
 6   total sulfur dioxide  229 non-null    float64
 7   density               229 non-null    float64
 8   pH                    229 non-null    float64
 9   sulphates             229 non-null    float64
 10  alcohol               229 non-null    float64
dtypes: float64(11)
memory usage: 29.6 KB


In [38]:
X_test_median_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229 entries, 158 to 966
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         229 non-null    float64
 1   volatile acidity      229 non-null    float64
 2   citric acid           229 non-null    float64
 3   residual sugar        229 non-null    float64
 4   chlorides             229 non-null    float64
 5   free sulfur dioxide   229 non-null    float64
 6   total sulfur dioxide  229 non-null    float64
 7   density               229 non-null    float64
 8   pH                    229 non-null    float64
 9   sulphates             229 non-null    float64
 10  alcohol               229 non-null    float64
dtypes: float64(11)
memory usage: 29.6 KB


Получение метрики MAE для оценки заполненных пропусков:

In [39]:
mae_with_mean = mean_absolute_error(X_test[features_with_nan].where(mask[features_with_nan]).stack().to_numpy(), X_test_mean_filled[features_with_nan].where(mask[features_with_nan]).stack().to_numpy())
mae_with_median = mean_absolute_error(X_test[features_with_nan].where(mask[features_with_nan]).stack().to_numpy(), X_test_median_filled[features_with_nan].where(mask[features_with_nan]).stack().to_numpy())

In [40]:
print(mae_with_mean)
print(mae_with_median)

3.1913510495581496
2.944393685258964


#### Импутация пропусков на основе **условного распределения многомерной нормальной модели**:

In [41]:
X_test_gauss_filled = X_test_with_nan.copy() # подготовка датасета

In [42]:
m = np.mean(X_train, axis=0) # вектор средних по признакам
sigma = np.cov(X_train, rowvar=False) # ковариационная матрица между признаками
cols = list(X_train.columns) # сохранение порядка столбцов

In [44]:
n_rows, n_features = X_test_gauss_filled.to_numpy(dtype=float).shape # сохранение размеров тестовой матрицы

In [45]:
for i in range(n_rows):
  row = X_test_gauss_filled.to_numpy(dtype=float)[i, :] # берем i-ю строку
  miss_mask = np.isnan(row) # находим, какие признаки в этой строке пропущены
  miss_idx = np.where(miss_mask)[0] # индексы пропущенных признаков
  obs_idx = np.where(~miss_mask)[0] # индексы наблюдаемых признаков
  if obs_idx.size == 0: # если в строке нет ни одного наблюдаемого признака, то восстановить по условному распределению нечем - используем безусловные средние m
    row[miss_idx] = m[miss_idx]
    continue
  x_obs = row[obs_idx] # формирование наблюдаемой части вектора x_obs
  m_obs = m[obs_idx] # и соответствующие средние m_obs
  m_miss = m[miss_idx] # средние по пропущенным компонентам
  sigma_oo = sigma[np.ix_(obs_idx, obs_idx)] # ковариация наблюдаемых признаков между собой
  sigma_mo = sigma[np.ix_(miss_idx, obs_idx)] # ковариация пропущенных признаков с наблюдаемыми
  sigma_oo = sigma_oo + eps * np.eye(sigma_oo.shape[0]) # регуляризированная матрица
  Z = np.linalg.solve(sigma_oo, (x_obs - m_obs)) # решение системы
  row[miss_idx] = m_miss + sigma_mo @ Z # вычисление условного ожидания для пропущенных компонент и запись их на места пропусков
  X_test_gauss_filled.to_numpy(dtype=float)[i, :] = row # сохранение обновленной строки


/tmp/ipython-input-3527190540.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m_obs = m[obs_idx]
/tmp/ipython-input-3527190540.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m_miss = m[miss_idx]


In [46]:
X_test_gauss_filled.loc[:, :] = X_test_gauss_filled.to_numpy(dtype=float) # обновление датафрейма

Получение метрики MAE для оценки заполненных пропусков:

In [47]:
mae_with_gauss = mean_absolute_error(X_test[features_with_nan].where(mask[features_with_nan]).stack().to_numpy(), X_test_gauss_filled[features_with_nan].where(mask[features_with_nan]).stack().to_numpy())

In [48]:
mae_with_gauss

2.0877625297985563

### Вывод:  
Методы заполнения средним и медианой вновь показали сопоставимый уровень ошибки, что ожидаемо: оба подхода являются независимой по признакам константной импьютацией и не используют межпризнаковые зависимости. Медиана продемонстрировала немного лучший результат, что может быть связано с меньшей чувствительностью к выбросам.

Импьютация с использованием многомерной гауссовской модели существенно снизила ошибку восстановления (примерно на 30–35% относительно медианы и более чем на 1 единицу MAE относительно среднего). Это подтверждает, что при одновременном наличии пропусков в нескольких признаках использование ковариационной структуры данных становится ещё более важным. Модель учитывает линейные зависимости между всеми наблюдаемыми и пропущенными переменными и восстанавливает значения согласованно с общей структурой распределения.

Таким образом, при наличии корреляций между признаками многомерная вероятностная модель обеспечивает заметно более точную и согласованную импьютацию по сравнению с независимым заполнением константами.

## 3) Пропуски в таргете


Формирование объединенной матрицы для оценки параметров совместного многомерного нормального распределения:

In [63]:
Z_train = pd.concat([X_train, y_train], axis=1)

In [64]:
m = np.mean(Z_train, axis=0) # вектор средних по всем переменным (сначала по признакам, потом по таргету)
sigma = np.cov(Z_train, rowvar=False) # ковариационная матрица

In [66]:
m_X = m[:-1] # среднее по признакам
m_y = m[-1] # среднее по таргету

/tmp/ipython-input-28169041.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m_y = m[-1]


In [67]:
sigma_XX = sigma[:-1, :-1] # ковариация между признаками
sigma_yX = sigma[-1, :-1] # ковариация таргета с признаками
sigma_yy = sigma[-1, -1] # дисперсия таргета

In [68]:
X_test = X_test.to_numpy()
m_X = np.asarray(m_X)
m_y = float(m_y)

In [73]:
sigma_XX_reg = sigma_XX + eps * np.eye(sigma_XX.shape[0]) # регуляризация ковариации признаков для устойчивости
beta = np.linalg.solve(sigma_XX_reg, sigma_yX.T).T # решение системы

In [74]:
y_pred_gauss = m_y + (X_test - m_X) @ beta.T # вычисление условного математического ожидания таргета

Получение метрики MAE для оценки заполненных пропусков:

In [75]:
mae_gauss = mean_absolute_error(y_test, y_pred_gauss)

In [76]:
mae_gauss

0.4773030310829565

Линейная регрессия:


In [78]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test) # получение предсказаний таргета от линейной регрессии

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Получение метрики MAE для оценки заполненных пропусков:

In [79]:
mae_lr = mean_absolute_error(y_test, y_pred_lr)

In [80]:
mae_lr

0.47733983525886087

#### Вывод:  
Разница между моделями составляет менее
$4 * 10^-5  $
, что практически нулевое расхождение с точки зрения численной точности. Это ожидаемый результат: при предположении о совместном многомерном нормальном распределении переменных условное математическое ожидание
$
\mathbb{E}[y \mid X]
$ является линейной функцией признаков. Соответствующие коэффициенты выражаются через ковариационные блоки:
$$
\beta
=
\Sigma_{XX}^{-1}\Sigma_{Xy}
$$

что приводит к той же линейной зависимости, которую оценивает классическая линейная регрессия методом наименьших квадратов.

Таким образом, эксперимент подтверждает теоретический факт: при гауссовских предположениях генеративная модель (оценка совместного распределения) и дискриминативная модель (линейная регрессия) дают эквивалентные предсказания.